# ANOVA de un factor

*¿Para qué se utiliza?*
El Análisis de Varianza (ANOVA) es una técnica estadística utilizada para comparar las medias de dos o más grupos y determinar si existen diferencias significativas entre ellas. Aunque el t de Student puede compararse con ANOVA cuando solo hay dos grupos, el ANOVA es especialmente útil cuando se comparan tres o más.

*¿Cómo funciona?*
El ANOVA evalúa la proporción de la variabilidad total de los datos que se debe a diferencias entre los grupos (causadas por la variable independiente) y la que se debe a diferencias dentro de los grupos (variabilidad aleatoria). Esta relación se resume en el estadístico F. Un valor F grande sugiere que hay más variabilidad explicada por el grupo al que pertenece cada observación que la atribuible al azar.

*Tipos de ANOVA:*
- ANOVA de un factor (one-way ANOVA): Compara la media de una variable cuantitativa entre niveles de un solo factor categórico (por ejemplo, ingreso promedio por nivel educativo).
- ANOVA de dos o más factores (factorial ANOVA): Evalúa simultáneamente el efecto de varios factores independientes y sus posibles interacciones.
- ANCOVA (Análisis de Covarianza): Extiende el ANOVA al incluir covariables cuantitativas para controlar efectos de otras variables.
- MANOVA (Análisis Multivariado de Varianza): Se utiliza cuando hay dos o más variables dependientes que se analizan de forma conjunta.

*Variables consideradas:*
- Una variable dependiente cuantitativa (de intervalo o razón).
- Una o más variables independientes (factores) categóricas con dos o más niveles (grupos o condiciones).

*Hipótesis planteadas (para ANOVA de un factor):*
- Hipótesis nula (H₀): Todas las medias poblacionales son iguales.
- Hipótesis alternativa (H₁): Al menos una de las medias es diferente.

*Supuestos o requisitos principales:*
- Independencia: Las observaciones dentro y entre los grupos deben ser independientes.
- Normalidad: La variable dependiente debe estar distribuida normalmente dentro de cada grupo.
- Homogeneidad de varianzas: Las varianzas de la variable dependiente deben ser aproximadamente iguales en todos los grupos.
Nota: El ANOVA es robusto ante desviaciones moderadas de los supuestos de normalidad y homogeneidad de varianzas, especialmente cuando los tamaños muestrales son similares entre grupos.

*Criterio de decisión:*
Si el valor p asociado al estadístico F es menor que el nivel de significancia (por ejemplo, α = 0.05), se rechaza la hipótesis nula y se concluye que al menos una media difiere. Sin embargo, el ANOVA no indica cuál o cuáles grupos difieren. Para eso, deben aplicarse pruebas post-hoc (como Tukey, Bonferroni o Scheffé) o realizar comparaciones específicas planificadas.


**Caso de uso**
Una cadena de comida rápida planea agregar un nuevo producto a su menú, pero están indecisos entre tres posibles campañas de mercadotecnica. Como un experimento, el producto fue introducido en varias ubicaciones seleccionadas aleatoriamente utilizando diferentes campañas. Se registraron las ventas del nuevo producto por las primeras cuatro semanas. El archivo "marketing.csv" contiene las siguientes variables:  
- MarketID: identificador del mercado
- MarketSize: tamaño del mercado de acuerdo a las ventas
- LocationID: identificador de la ubicación de la tienda
- AgeOfStore: antigüedad de la tienda en años
- Promotion: promoción (1 de 3) que fue probada
- week: semana en que se llevó a cabo la promoción.
- SalesInThousands: ventas para una ubicación específica (LocationID), promoción (Promotion) y semana (week).

Para comparar las ventas de acuerdo a las diferentes promociones, utilizaremos una ANOVA  
Hipótesis nula: todas las medias son iguales.  
Hipótesis alternativa: al menos un par es diferente.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('https://github.com/adan-rs/amd/raw/main/data/marketing.csv')

In [ ]:
df.info()

In [ ]:
df['Promotion'].value_counts()

In [ ]:
df.sample(4)

## Supuesto de igualdad de varianzas

In [ ]:
from scipy.stats import levene

def compare_variances(data, group_column, measure_column, groups=None, alpha=0.05):
    """
    Realiza e interpreta una prueba de Levene para comparar varianzas entre múltiples grupos.   
    Args:
        data: DataFrame con los datos
        group_column: Nombre de la columna que identifica los grupos
        measure_column: Nombre de la columna con la variable a comparar
        groups: Lista de grupos a comparar (si es None, usa todos los grupos encontrados)
        alpha: Nivel de significancia (default: 0.05)
    Returns:
        p-value
    """
    # Identificar grupos si no se especifican
    grupos_disponibles = data[group_column].unique()
    if len(grupos_disponibles) < 2:
        raise ValueError(f"La columna {group_column} debe tener al menos dos grupos")
    
    # Usar grupos especificados o todos los disponibles
    grupos = groups if groups is not None else grupos_disponibles
    
    # Verificar que todos los grupos especificados existen
    grupos_invalidos = set(grupos) - set(grupos_disponibles)
    if grupos_invalidos:
        raise ValueError(f"Grupos no encontrados en los datos: {grupos_invalidos}")
    
    # Obtener datos de cada grupo
    datos_grupos = [data[data[group_column] == grupo][measure_column] for grupo in grupos]
    
    # Calcular estadísticas por grupo
    stats = pd.DataFrame({
        'n': [len(datos) for datos in datos_grupos],
        'varianza': [datos.var() for datos in datos_grupos],
        'desv_std': [datos.std() for datos in datos_grupos]
    }, index=grupos)
    
    # Realizar prueba de Levene
    stat, p_value = levene(*datos_grupos)
    
    # Preparar interpretación
    es_significativo = p_value < alpha
    interpretacion = (f"Prueba de Levene para {len(grupos)} grupos\n"
                     f"Estadístico de Levene: {stat:.4f}\n"
                     f"Valor p: {p_value:.4f}\n\n"
                     "Estadísticas por grupo:\n")
    
    # Añadir estadísticas de cada grupo
    for grupo in grupos:
        n = stats.loc[grupo, 'n']
        var = stats.loc[grupo, 'varianza']
        std = stats.loc[grupo, 'desv_std']
        interpretacion += f"{grupo}: n={n}, varianza={var:.2f}, desv. estándar={std:.2f}\n"
    
    # Añadir conclusión
    interpretacion += f"\nConclusión: {'Se rechaza' if es_significativo else 'No se rechaza'} "
    interpretacion += "la hipótesis nula de igualdad de varianzas"
    interpretacion += (" (hay diferencias significativas en las varianzas entre los grupos)" 
                      if es_significativo 
                      else "\n (no hay evidencia de diferencias significativas en las varianzas)")
    
    print(interpretacion)
    return p_value

In [ ]:
resultados_levene = compare_variances(df,'Promotion', 'SalesInThousands')

## Supuesto de normalidad

In [ ]:
from scipy import stats
import pandas as pd

def test_ks_normality(df, group_column, measure_column):
    """
    Realiza la prueba de Kolmogorov-Smirnov para evaluar normalidad por grupos
    Args:
        df: DataFrame con los datos
        group_col: Nombre de la columna de agrupación
        value_col: Nombre de la columna con valores a evaluar
    Returns:
        DataFrame con resultados de la prueba por grupo
    """
    results = []
    for group in df[group_column].unique():
        data = df[df[group_column] == group][measure_column]
        stat, pval = stats.kstest(data, 'norm', args=(data.mean(), data.std()))
        results.append({'grupo': group,'estadistico_ks': stat.round(4),'valor_p': pval.round(4)})
        
    return pd.DataFrame(results)

In [ ]:
normalidad = test_ks_normality(df,'Promotion', 'SalesInThousands')
normalidad

Sin embargo, varios estudios muestran que la prueba ANOVA es robusta ante violaciones de la normalidad si:
- Los grupos tienen un tamaño similar
- Hay por lo menos 40 observaciones en cada grupo.

## ANOVA

In [ ]:
from scipy.stats import f_oneway

def realizar_anova(df, group_column, measure_column, alpha=0.05):
    """
    Realiza prueba ANOVA de una vía para comparar medias entre grupos.
    Parámetros:
    df (pandas.DataFrame): DataFrame con los datos
    grupo_col (str): Nombre de la columna que contiene los grupos
    valor_col (str): Nombre de la columna con los valores a comparar
    alpha (float): Nivel de significancia (default 0.05)
    Retorna:
    p-value
    """
    # Crear valores por grupo
    grupos = df[group_column].unique()
    valores_grupos = [df[df[group_column] == grupo][measure_column] for grupo in grupos]
    
    # Realizar prueba ANOVA
    statistic, p_value = f_oneway(*valores_grupos)
    
    # Determinar resultado
    resultado = "Se rechaza la hipótesis nula: hay al menos una diferencia significativa entre los grupos." if p_value < alpha else "No se puede rechazar la hipótesis nula: no hay suficiente evidencia para afirmar que hay diferencias significativas entre los grupos."

    print(f'Estadistico {statistic:.4f}')
    print(f'Valor p: {p_value:.4f}')
    print(resultado)
    
    return  p_value

In [ ]:
resultado = realizar_anova(df,'Promotion', 'SalesInThousands')

# Pruebas post-hoc
La tabla ANOVA no indica qué grupo es diferente al resto, sin embargo, las pruebas post-hoc son útiles para detectar qué grupo es diferente al resto. 
- LSD/DMS (Diferencia menos significativa): Es el equivalente a múltiples pruebas t, no se hacen correcciones y los resultados no son precisos.
- Bonferroni: Corrige el nivel de significancia dividiéndolo entre el número de grupos. Es preferible cuando son pocas comparaciones. 
- Tukey: Preferible cuando son muchas comparaciones. Es deseable que el tamaño de cada grupo sea igual. 
- REGWQ (Ryan-Einot-Gabriel-Welsh): Recomendable, pero se debe evitar cuando las muestras son de diferente tamaño.
- Dunnett: Es apropiada cuando se desea comparar con un grupo de control.
- Gabriel: Apropiada cuando el tamaño de las muestras es ligeramente diferente.
- GT2 de Hochberg: Apropiadas cuando el tamaño de las muestras es muy diferente
- Games- Howell: Recomendable cuando las varianzas son diferentes.

La librería statsmodel permite realizar la prueba de Tukey, Bonferroni y Dunnet

In [ ]:
def post_hoc_test(data, group_column, value_column, test_type='tukey', control_group=None, alpha=0.05):
    """
    Realiza pruebas post hoc múltiples: Tukey o Bonferroni.
    Parámetros:
    data: DataFrame que contiene los datos
    group_column: Nombre de la columna que contiene los grupos
    value_column: Nombre de la columna que contiene los valores a comparar
    test_type: Tipo de prueba ('tukey', 'bonferroni')
    alpha : Nivel de significancia (por defecto 0.05)
    Retorna:
    Objeto con resultados de la prueba seleccionada
    """
    from statsmodels.stats.multicomp import MultiComparison
    from statsmodels.stats.multicomp import pairwise_tukeyhsd
    
    mc = MultiComparison(data[value_column], data[group_column])
    
    if test_type.lower() == 'tukey':
        print(mc.tukeyhsd(alpha=alpha))
    
    elif test_type.lower() == 'bonferroni':
        print(mc.allpairtest(stats.ttest_ind, method='bonf')[0])
    
    else:
        raise ValueError("Tipo de prueba no válido. Usar 'tukey' o 'bonferroni'")

In [ ]:
resultado_tukey = post_hoc_test(df, "Promotion", "SalesInThousands", test_type='tukey')
resultado_tukey

In [ ]:
resultado_bonferroni = post_hoc_test(df, "Promotion", "SalesInThousands", test_type='bonferroni')
resultado_bonferroni

Ejemplo de redacción de conclusiones: 

>Se realizó un ANOVA de un factor para analizar cómo ________ influye en _________. Los resultados muestran que _________tiene un efecto significativo en __________, F(_,_) = ___, p = ___. Las comparaciones post-hoc utilizando el método de Tukey HSD indican que la media de ______ es significativamente diferente a la media de _________ y _______

## Prueba Kruskal-Wallis
*¿Para qué se utiliza?*
La prueba de Kruskal-Wallis es una alternativa no paramétrica al análisis de varianza (ANOVA) de un factor. Se utiliza para comparar si existen diferencias significativas entre tres o más grupos independientes, sin asumir que los datos siguen una distribución normal. Es útil cuando los supuestos del ANOVA no se cumplen, especialmente en presencia de datos asimétricos, ordinales o con valores atípicos.

*¿Cómo funciona?*
En lugar de comparar medias, como lo hace el ANOVA, esta prueba compara las posiciones (rangos) que ocupan los datos en el conjunto total. Los valores de todas las observaciones se ordenan de menor a mayor y se les asignan rangos. Luego, se evalúa si los rangos promedio difieren entre los grupos.

*Variables consideradas:*
- Una variable dependiente ordinal o cuantitativa (de intervalo o razón, pero no necesariamente normal).
- Una variable independiente categórica con tres o más grupos independientes.

*Hipótesis planteadas:*
- Hipótesis nula (H₀): Las distribuciones (o medianas) de los grupos son iguales.
- Hipótesis alternativa (H₁): Al menos uno de los grupos tiene una distribución (o mediana) diferente.

*Supuestos o requisitos principales:*
- Independencia: Las observaciones deben ser independientes dentro y entre los grupos.
- Escala adecuada: La variable dependiente debe ser al menos ordinal (o cuantitativa continua sin normalidad).
- Distribuciones similares: Se asume que las distribuciones de los grupos tienen forma similar si se desea interpretar la prueba como una comparación de medianas.

*Criterio de decisión:*
Se calcula un estadístico H, que se aproxima a una distribución chi-cuadrada con k−1 grados de libertad, donde k es el número de grupos.
- Si el valor p es menor que el nivel de significancia (por ejemplo, α = 0.05), se rechaza la hipótesis nula y se concluye que al menos un grupo difiere de los demás.
- Si el valor p es mayor que 0.05, no se rechaza la hipótesis nula.
Nota: La prueba indica si hay diferencias globales, pero no especifica cuáles grupos difieren. Para ello, se deben aplicar pruebas post-hoc no paramétricas, como las comparaciones múltiples de Dunn con ajuste por Bonferroni.


In [ ]:
from scipy.stats import kruskal
import pandas as pd

def kruskal_test(df: pd.DataFrame, group_col: str, value_col: str, alpha: float = 0.05) -> dict:
    """
    Realiza la prueba de Kruskal-Wallis para comparar múltiples grupos.
    Args:
        df: DataFrame con los datos
        group_col: Nombre de la columna que contiene los grupos
        value_col: Nombre de la columna con los valores a comparar
        alpha: Nivel de significancia (por defecto 0.05)  
    Returns:
        dict: Diccionario con los resultados de la prueba
    """
    # Obtener grupos únicos
    grupos = [df[df[group_col] == g][value_col] for g in df[group_col].unique()]
    
    # Realizar prueba Kruskal-Wallis
    statistic, p_value = kruskal(*grupos)
    
    # Preparar resultados
    resultados = {
        'estadistico': statistic,
        'p_valor': p_value,
        'alfa': alpha,
        'decision': 'Se rechaza H0' if p_value < alpha else 'No se rechaza H0',
        'interpretacion': ('Hay evidencia de diferencias significativas entre los grupos.' 
                         if p_value < alpha 
                         else 'No hay suficiente evidencia para afirmar diferencias significativas entre los grupos.')}
    
    return resultados

Ejemplo de un reporte de resultados
>Se realizó una prueba de Kruskal-Wallis con el objetivo de comparar la rentabilidad mensual (en porcentaje) de tres tipos de portafolios de inversión: conservador, moderado y agresivo.
La muestra consistió en 20 mediciones mensuales por cada tipo de portafolio. Dado que los datos no seguían una distribución normal (según la prueba de Shapiro-Wilk), se optó por esta prueba no paramétrica.
El resultado fue H(2) = 7.62, con un valor p = 0.0221, lo que indica que existe una diferencia estadísticamente significativa en la mediana de rentabilidad entre al menos dos de los portafolios.


In [ ]:
# Ejemplo de uso:
resultados = kruskal_test(df, 'Promotion', 'SalesInThousands')
print(f"Estadístico: {resultados['estadistico']:.4f}")
print(f"Valor p: {resultados['p_valor']:.4f}")
print(f"Decisión: {resultados['decision']}")
print(f"Interpretación: {resultados['interpretacion']}")